<a href="https://colab.research.google.com/github/xcollantes/stock-quant-frontend/blob/main/stock_fundamentals_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install altair==5.0.1 # Needed for yOffset of bar charts

import altair as alt
import enum
import pandas as pd
import requests
import urllib
import yfinance as yf
import csv

from altair import datum
from io import StringIO
from IPython.display import display
from IPython.display import Markdown as md

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

ALPHAVANTAGE_API: str = "6I7UILQC83K8JZ0Z"
FMG_KEY: str = "6fc03c9a2f6330be1eff2950bf3e6f0c"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


# Find stocks in the market which have potential for investing long term

A point system will decide if a stock is a good investment.

- Sharp drops: +20
- Beat earnings in last 4 quarters: +10
- Earnings per share: +5 per dollar
- Dividends paid out: +0 (not considered in this analysis)

## Find stocks with sharp drops

Use cases include events where a stock has a shock to it's market price due to some event such as hype from some news to the company, industry events, company going bankrupt, or some other reason.

**Methodology:** Look at the previous close of asset, compare if the asset at close

**Drawbacks:** Metrics are taken at the close price to next close price so fluctuations within a day will not be detected.


Data sources

- yFinance for symbol data
- alphavantage.co for advanced symbol data
    - Limit: 5 / min; 500 / day
- api.nasdaq.com for list of NASDAQ stocks

- financialmodelingprep.com for industry data
    - Limit: 250 calls / day

In [2]:
# @title Stock symbol functions

def clean_symbol(user_in: str) -> str:
  """Parse user input for one stock symbol and return symbol."""
  return user_in.strip().upper()


def clean_symbols_many(user_in: str) -> list[str]:
  """Parse user input for many stock symbols and return list of symbols."""
  if user_in == "":
    return [""]

  split_in: list[str] = user_in.split(",")
  return [ x.strip().upper() for x in split_in if x.strip() != "" ]


def get_symbol_data(ticker_symbol: str, days_ago: int) -> pd.DataFrame:
  """Given a date range, returns historical price range.

  Args:
    ticker_symbol: String of ticker.
    days_ago: Range of stock history prior to today.

  Returns:
    Historical data.
  """
  df: pd.DataFrame = yf.Ticker(ticker_symbol.upper())
  history = df.history(period=f"{days_ago}d")
  history["Date NY"] = history.index  # Add non-index field
  history.sort_values(by="Date", ascending=False)
  return history


def has_large_drop_last3days(symbol_data: pd.DataFrame) -> bool:
  """Return True if there is a spike in price.

  Standard deviation is a calculation of last some date range in the given
  dataset (recommendation of last 3 months) but the determination of
  spike is only done on the last 3 days.

  Args:
    symbol_data: DataFrame with at least date and close prices for a symbol.

  Returns:
    True if the change between two dates is greater than one standard deviation.
  """

  close_std_dev: float = symbol_data.std()["Close"]

  top_dates: int = 3
  prev_day: int = 0

  for index, day in symbol_data.iterrows():
    if prev_day != 0:
      variance: float = day["Close"] - prev_day
      if abs(variance) > close_std_dev * 2:
        return True
    top_dates -= 1

    prev_day = day["Close"]

  return False


def multiple_symbols_chart(symbol_data: pd.DataFrame, title: str = "") -> alt.Chart:
  """Show multi-line graph for a many stock symbols."""
  x_axis = alt.X("Date NY:T", axis=alt.Axis(labelAngle=-50), title="Dates")
  y_axis = alt.Y("PctChange", title="Percent change", axis=alt.Axis(labelExpr="datum.value * 100 + '%'"))

  color = alt.Color("Name:N")
  selection = alt.selection_multi(fields=["Name"], bind="legend", on='mouseover', toggle='event.ctrlKey')
  selection_opacity = alt.condition(selection, alt.value(1), alt.value(0.09))
  tooltip = [alt.Tooltip("Name:N"),
             alt.Tooltip("Symbol:N"),
             alt.Tooltip("Date NY:T"),
             alt.Tooltip("Close", format="$.2f")]
  point = alt.OverlayMarkDef(filled=False, fill="white")

  change_chart = alt.Chart(symbol_data).mark_line(point=point).encode(
      x=x_axis,
      y=y_axis,
      color=color,
      tooltip=tooltip,
      opacity=selection_opacity
  ).properties(
    title={"text": "Industry prices changes", "subtitle": "Hold SHIFT to select multiple in Legend"},
    height=500,
    width=1100,
    # selection=alt.selection_multi(fields=[''], )
  ).add_selection(selection).interactive()

  dollar_chart = alt.Chart(symbol_data).mark_line(point=point).encode(
      x=x_axis,
      y=alt.Y("Close", title="Closing price", axis=alt.Axis(labelExpr="'$' + datum.value")),
      color=color,
      tooltip=tooltip,
      opacity=selection_opacity
  ).properties(
    height=200,
    width=1100,
  ).add_selection(selection).interactive()

  return alt.vconcat(change_chart, dollar_chart).configure_axis(
    labelFontSize=18,
    titleFontSize=18
  ).configure_legend(
      labelFontSize=15,
      titleFontSize=12
  ).configure_title(
      fontSize=20
  ).configure_point(
    size=200)


def earnings_beat_chart(earnings_df: pd.DataFrame, symbol_name: str = ""):
  date_filter: str = "year(datum.date) > year(now()) - 5"

  if symbol_name != "":
    symbol_name = f"({symbol_name})"

  expected_chart = alt.Chart(earnings_df).mark_point(
      size=1000, strokeWidth=7, color="gray").encode(
      x=alt.X("date:T"),
      y=alt.Y("estimatedEarning:Q"),
      tooltip=alt.Tooltip(["date:T", "estimatedEarning:Q",
                           "actualEarningResult:Q"]),
  ).transform_filter(date_filter).properties(title=f"Earnings beat quarterly {symbol_name}",
               height=500,
               width=1100)

  actual_chart = alt.Chart(earnings_df).mark_point(
      size=1000, strokeWidth=7, color="green").encode(
      x=alt.X("date:T"),
      y=alt.Y("actualEarningResult:Q"),
      tooltip=alt.Tooltip(["date:T", "estimatedEarning:Q",
                           "actualEarningResult:Q"]),
  ).transform_filter(date_filter)

  return (expected_chart + actual_chart).configure_axis(
    labelFontSize=18,
    titleFontSize=18
  ).configure_legend(
      labelFontSize=15,
      titleFontSize=12
  ).configure_title(
      fontSize=20
  ).configure_point(
    size=200)


def symbol_show_line(symbol_data: pd.DataFrame, title: str = "") -> alt.Chart:
  """Show line graph for a single stock symbol."""
  y_axis = alt.Y("Close", title="Close prices")

  chart = alt.Chart(symbol_data).mark_line(point=True).encode(
      x=alt.X("Date NY:T", title="Dates"),
      y=y_axis
  ).properties(
    title=title,
    width=900
  )

  upper_std = alt.Chart(symbol_data).mark_rule(
      color="green",
      strokeDash=(6, 2)).encode(
      y=alt.Y("Std_plus:Q"),
  )

  line = alt.Chart(symbol_data).mark_rule(
      color="red",
      strokeWidth=2,
      strokeDash=(5, 2)).encode(
      y=alt.Y("mean(Close):Q"),
  )

  lower_std = alt.Chart(symbol_data).mark_rule(
      color="blue",
      strokeDash=(6, 2)).encode(
      y=alt.Y("Std_minus:Q"),
  )

  return (chart + upper_std + line + lower_std).configure_axis(
    labelFontSize=18,
    titleFontSize=18
  ).configure_title(fontSize=20)


def show_line_chart(df: pd.DataFrame, x: str, y: str, title: str = "") -> alt.Chart:
  """Show line graph for income statement for a symbol."""
  chart = alt.Chart(df).mark_line(point=True).encode(
      x=alt.X(f"{x}:T"),
      y=alt.Y(f"{y}:Q"),
  ).properties(
    title=title,
    width=900
  )

  return (chart).configure_axis(
    labelFontSize=18,
    titleFontSize=18
  ).configure_title(fontSize=20)


def _get_alphavantage_url(symbol: str, statement_type: str) -> str:
  """Get URL for API."""
  return (f"https://www.alphavantage.co/query?function={statement_type}" +
          f"&symbol={symbol}" +
          f"&apikey={ALPHAVANTAGE_API}")


def get_symbol_income_statement(symbol: str) -> pd.DataFrame:
  """Return DataFrame of income statement for a company."""
  url = _get_alphavantage_url(symbol, "INCOME_STATEMENT")
  response: requests.Response = requests.get(url)
  income_stmt_df: pd.DataFrame = pd.json_normalize(response.json(), record_path=["annualReports"])
  income_stmt_df.sort_values(by="fiscalDateEnding")
  return income_stmt_df


def get_earnings(symbol: str) -> pd.DataFrame:
  url = _get_alphavantage_url(symbol, "EARNINGS")
  response = requests.Response = requests.get(url)
  # print(response.json())
  return pd.json_normalize(response.json(), record_path="annualEarnings")


def get_balance_sheet(symbol: str) -> pd.DataFrame:
  url = _get_alphavantage_url(symbol, "BALANCE_SHEET")
  response = requests.Response = requests.get(url)
  return pd.json_normalize(response.json(), record_path="annualReports")


def get_cash_flow(symbol: str) -> pd.DataFrame:
  url = _get_alphavantage_url(symbol, "CASH_FLOW")
  response = requests.Response = requests.get(url)
  # print(response.json())
  return pd.json_normalize(response.json(), record_path="annualReports")


def get_balance_sheet(symbol: str) -> pd.DataFrame:
  url = _get_alphavantage_url(symbol, "BALANCE_SHEET")
  response = requests.Response = requests.get(url)
  # print(response.json())
  return pd.json_normalize(response.json(), record_path="annualReports")


def get_earnings_surprises(symbol: str) -> pd.DataFrame:
  url = f"https://financialmodelingprep.com/api/v3/earnings-surprises/{symbol}?apikey={FMG_KEY}"
  response = requests.Response = requests.get(url)
  return pd.json_normalize(response.json())


def get_ratios_ttm(symbol: str) -> dict:
  url = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{symbol}?apikey={FMG_KEY}"
  response = requests.Response = requests.get(url)
  return pd.json_normalize(response.json())

def get_overview(symbol: str) -> dict:
  """Return JSON of company stats and description."""
  url = _get_alphavantage_url(symbol, "OVERVIEW")
  response = requests.Response = requests.get(url)
  # print(response.json())
  return response.json()


# get_symbol_income_statement("V")
# get_balance_sheet("V")
# get_cash_flow("V")
# get_earnings("V")
# get_overview("V")
# get_earnings_surprises("V")
# earnings_beat_chart(earnings_df)

out = clean_symbols_many("v,MSFT ,,GOOG, ,")
assert out == ["V", "MSFT", "GOOG"], f"{out} not equal"
out = clean_symbols_many("")
assert out == [""], f"{out} not equal"
out = clean_symbols_many("V")
assert out == ["V"], f"{out} not equal"

In [3]:
# @title Get DataFrame of stock symbols and metadata
response_csv = requests.get("https://raw.githubusercontent.com/xcollantes/stock_analysis_dataset/main/companies.csv")
companies_file = StringIO(response_csv.text)
file_df = pd.read_csv(companies_file)
symbols_df = file_df.drop(["logo"], axis=1)

In [4]:
symbols_df

,ticker,company name,short name,industry,description,website,ceo,exchange,market cap,sector,tag 1,tag 2,tag 3
0,A,Agilent Technologies Inc.,Agilent,Medical Diagnostics & Research,Agilent Technologies Inc is engaged in life sc...,http://www.agilent.com,Michael R. McMullen,New York Stock Exchange,2.421807e+10,Healthcare,Healthcare,Diagnostics & Research,Medical Diagnostics & Research
1,AA,Alcoa Corporation,Alcoa,Metals & Mining,Alcoa Corp is an integrated aluminum company. ...,http://www.alcoa.com,Roy Christopher Harvey,New York Stock Exchange,5.374967e+09,Basic Materials,Basic Materials,Aluminum,Metals & Mining
2,AABA,Altaba Inc.,Altaba,Asset Management,"Altaba Inc is an independent, non-diversified,...",http://www.altaba.com,Thomas J. Mcinerney,Nasdaq Global Select,4.122368e+10,Financial Services,Financial Services,Asset Management,NaN
3,AAC,AAC Holdings Inc.,AAC,Health Care Providers,AAC Holdings Inc provides inpatient and outpat...,http://www.americanaddictioncenters.org,Michael T. Cartwright,New York Stock Exchange,6.372010e+07,Healthcare,Healthcare,Medical Care,Health Care Providers
4,AADR,AdvisorShares Dorsey Wright ADR,AdvisorShares Dorsey Wright,NaN,The investment seeks long-term capital appreci...,http://www.advisorshares.com,NaN,NYSE Arca,1.031612e+08,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,ZTS,Zoetis Inc. Class A,Zoetis,Drug Manufacturers,Zoetis Inc is a developer and manufacturer of ...,http://www.zoetis.com,Juan Ramon Alaix,New York Stock Exchange,4.205627e+10,Healthcare,Healthcare,Drug Manufacturers - Specialty & Generic,Drug Manufacturers
6364,ZUMZ,Zumiez Inc.,Zumiez,Retail - Apparel & Specialty,Zumiez Inc is a multi-channel specialty retail...,http://www.zumiez.com,Richard M. Brooks,Nasdaq Global Select,6.150368e+08,Consumer Cyclical,Consumer Cyclical,Specialty Retail,Retail - Apparel & Specialty
6365,ZUO,Zuora Inc. Class A,Zuora,Application Software,Zuora Inc provides cloud-based software on a s...,https://www.zuora.com,Tien Tzuo,New York Stock Exchange,2.304595e+09,Technology,Technology,Software - Infrastructure,Application Software
6366,ZYME,Zymeworks Inc.,Zymeworks,Biotechnology,Zymeworks Inc is a clinical-stage biopharmaceu...,http://www.zymeworks.com,Ali Tehrani,New York Stock Exchange,5.042878e+08,Healthcare,Healthcare,Biotechnology,NaN


In [ ]:
# @title [DEPRECATE] Get list of stocks

headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0",
}

# Get latest daily data for stocks
response: requests.Response = requests.get(
    "https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=99999&exchange=nyse",
    headers=headers)
nasdaq_raw_df: pd.DataFrame = pd.json_normalize(
    response.json(),
    record_path=["data", "table", "rows"])
nasdaq_raw_df["pctchangeDecimal"] = nasdaq_raw_df["pctchange"].str.rstrip(
    "%").str.replace("--", "0").replace("", "0").astype("float") / 100

stock_industry_csv: requests.Response = requests.get(
    "https://raw.githubusercontent.com/xcollantes/stock_analysis_dataset/main/stock_industry.csv")
sectors_df: pd.DataFrame = pd.read_csv(
    StringIO(stock_industry_csv.content.decode("utf-8")))
nasdaq_df: pd.DataFrame = nasdaq_raw_df.merge(
    sectors_df[["symbol", "industry", "sector"]],
    how="left", left_on=["symbol"], right_on=["symbol"])

In [ ]:
# @title Compare closing prices changes by industry
compare_industry_name = "Coal" #@param ['Advertising & Marketing Services', 'Aerospace & Defense', 'Agriculture', 'Airlines', 'Application Software','Asset Management', 'Autos', 'Banks', 'Beverages - Alcoholic','Beverages - Non-Alcoholic', 'Biotechnology','Brokers & Exchanges', 'Building Materials', 'Business Services','Chemicals', 'Coal', 'Communication Equipment','Communication Services', 'Computer Hardware', 'Conglomerates','Consulting & Outsourcing', 'Consumer Packaged Goods','Credit Services', 'Drug Manufacturers', 'Education','Employment Services', 'Engineering & Construction','Entertainment', 'Farm & Construction Machinery','Forest Products', 'Health Care Plans', 'Health Care Providers','Homebuilding & Construction', 'Industrial Distribution','Industrial Products', 'Insurance', 'Insurance - Life','Insurance - Property & Casualty', 'Insurance - Specialty','Manufacturing - Apparel & Furniture', 'Medical Devices','Medical Diagnostics & Research', 'Medical Distribution','Medical Instruments & Equipment', 'Metals & Mining','Oil & Gas - Drilling', 'Oil & Gas - E&P','Oil & Gas - Integrated', 'Oil & Gas - Midstream','Oil & Gas - Refining & Marketing', 'Oil & Gas - Services','Online Media', 'Packaging & Containers', 'Personal Services','Publishing', 'REITs', 'Real Estate Services', 'Restaurants','Retail - Apparel & Specialty', 'Retail - Defensive','Semiconductors', 'Steel', 'Tobacco Products','Transportation & Logistics', 'Travel & Leisure','Truck Manufacturing', 'Utilities - Independent Power Producers','Utilities - Regulated', 'Waste Management'] {allow-input: true}
days_ago = 69 #@param {type:"slider", min:1, max:100, step:1}

agg_graph_df = pd.DataFrame()

symbol_name_df = symbols_df[symbols_df["industry"] == compare_industry_name][["ticker", "company name"]]
for index, row in symbol_name_df.iterrows():
  symbol = row["ticker"]
  name = row["name"]
  hist_symbol_df = get_symbol_data(symbol, days_ago)
  hist_symbol_df["ticker"] = symbol
  hist_symbol_df["Name"] = name
  agg_graph_df = pd.concat([agg_graph_df, hist_symbol_df])

# Add percent change column
sorted_df = agg_graph_df.sort_values(["ticker", "Date NY"]).reset_index(drop=True)
pct_change_series = sorted_df.groupby("ticker", group_keys=False, sort=False)["Close"].apply(lambda x: x.pct_change())

view_agg_pctchange_df = sorted_df
view_agg_pctchange_df["PctChange"] = pct_change_series

multiple_symbols_chart(view_agg_pctchange_df)

KeyError: ignored

In [ ]:
# @title Choose stocks to compare closing prices

days_ago = 11 #@param {type:"slider", min:1, max:100, step:1}
choose_symbols_many = "NET,IRNT,LOGI,BLK" #@param {type:"string"}

try:
  if choose_symbols_many == "":
    raise Exception("Enter comma separated list of stock symbols.")

  choose_symbols = clean_symbols_many(choose_symbols_many)

  agg_graph_df = pd.DataFrame()

  symbol_name_df = nasdaq_df[nasdaq_df["symbol"].isin(choose_symbols)][["symbol", "name"]]
  for index, row in symbol_name_df.iterrows():
    symbol = row["symbol"]
    name = row["name"]
    hist_symbol_df = get_symbol_data(symbol, days_ago)
    hist_symbol_df["Symbol"] = symbol
    hist_symbol_df["Name"] = name
    agg_graph_df = pd.concat([agg_graph_df, hist_symbol_df])

except Exception as e:
  print(e)

# Add percent change column
sorted_df = agg_graph_df.sort_values(["Symbol", "Date NY"]).reset_index(drop=True)
pct_change_series = sorted_df.groupby("Symbol", group_keys=False, sort=False)["Close"].apply(lambda x: x.pct_change())

view_agg_pctchange_df = sorted_df
view_agg_pctchange_df["PctChange"] = pct_change_series

multiple_symbols_chart(view_agg_pctchange_df)

/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)

In [ ]:
# @title P/E ratios
# @markdown Limited financialmodelingprep.com
choose_symbols_many = "NET,IRNT,LOGI,BLK" #@param {type:"string"}

choose_symbols = clean_symbols_many(choose_symbols_many)
choose_symbols

['NET', 'IRNT', 'LOGI', 'BLK']

In [ ]:
# @title Top drops
industry_name = "Software\u2014Infrastructure" #@param ['Advertising Agencies', 'Aerospace & Defense', 'Agricultural Inputs', 'Airlines', 'Airports & Air Services','Aluminum', 'Apparel Manufacturing', 'Apparel Retail', 'Asset Management', 'Auto & Truck Dealerships', 'Auto Manufacturers', 'Auto Parts', 'Banks—Diversified','Banks—Regional', 'Beverages—Brewers', 'Beverages—Non-Alcoholic','Beverages—Wineries & Distilleries', 'Biotechnology','Broadcasting', 'Building Materials','Building Products & Equipment', 'Business Equipment & Supplies','Capital Markets', 'Chemicals', 'Closed-End Fund - Equity','Coking Coal', 'Communication Equipment', 'Computer Hardware','Confectioners', 'Conglomerates', 'Consulting Services','Consumer Electronics', 'Copper', 'Credit Services','Department Stores', 'Diagnostics & Research', 'Discount Stores','Drug Manufacturers—General','Drug Manufacturers—Specialty & Generic','Education & Training Services', 'Electrical Equipment & Parts','Electronic Components', 'Electronic Gaming & Multimedia','Electronics & Computer Distribution','Engineering & Construction', 'Entertainment','Farm & Heavy Construction Machinery', 'Farm Products','Financial Conglomerates', 'Financial Data & Stock Exchanges','Food Distribution', 'Footwear & Accessories','Furnishings, Fixtures & Appliances', 'Gambling', 'Gold','Grocery Stores', 'Health Information Services','Healthcare Plans', 'Home Improvement Retail','Household & Personal Products', 'Independent Oil & Gas','Industrial Distribution', 'Information Technology Services','Insurance Brokers', 'Insurance—Diversified', 'Insurance—Life','Insurance—Property & Casualty', 'Insurance—Reinsurance','Insurance—Specialty', 'Integrated Freight & Logistics','Internet Content & Information', 'Internet Retail', 'Leisure','Lodging', 'Lumber & Wood Production', 'Luxury Goods','Marine Shipping', 'Medical Care Facilities', 'Medical Devices','Medical Distribution', 'Medical Instruments & Supplies','Metal Fabrication', 'Mortgage Finance', 'Oil & Gas Drilling','Oil & Gas E&P', 'Oil & Gas Equipment & Services','Oil & Gas Integrated', 'Oil & Gas Midstream','Oil & Gas Pipelines', 'Oil & Gas Refining & Marketing','Other Industrial Metals & Mining','Other Precious Metals & Mining', 'Packaged Foods','Packaging & Containers', 'Paper & Paper Products','Personal Services', 'Pharmaceutical Retailers','Pollution & Treatment Controls', 'Publishing', 'REIT—Diversified','REIT—Healthcare Facilities', 'REIT—Hotel & Motel','REIT—Industrial', 'REIT—Mortgage', 'REIT—Office','REIT—Residential', 'REIT—Retail', 'REIT—Specialty', 'Railroads','Real Estate Services', 'Real Estate—Development','Real Estate—Diversified', 'Recreational Vehicles','Rental & Leasing Services', 'Residential Construction','Resorts & Casinos', 'Restaurants','Scientific & Technical Instruments','Security & Protection Services','Semiconductor Equipment & Materials', 'Semiconductors','Shell Companies', 'Silver', 'Software—Application','Software—Infrastructure', 'Solar', 'Specialty Business Services','Specialty Chemicals', 'Specialty Industrial Machinery','Specialty Retail', 'Staffing & Employment Services', 'Steel','Telecom Services', 'Textile Manufacturing', 'Thermal Coal','Tobacco', 'Tools & Accessories', 'Travel Services', 'Trucking','Uranium', 'Utilities—Diversified','Utilities—Independent Power Producers','Utilities—Regulated Electric', 'Utilities—Regulated Gas','Utilities—Regulated Water', 'Utilities—Renewable','Waste Management'] {allow-input: true}
show_count = 35 #@param {type:"slider", min:1, max:100, step:1}

with pd.option_context("display.max_rows", None):
  if industry_name:
    display(symbols_df[symbols_df["industry"] == industry_name].sort_values(
      by="pctchangeDecimal", ascending=True).head(show_count))
  else:
    display(symbols_df.sort_values(
      by="pctchangeDecimal", ascending=True).head(show_count))

,symbol,name,lastsale,netchange,pctchange,marketCap,url,pctchangeDecimal,industry,sector
1272,SOS,SOS Limited American Depositary Shares,$5.91,-0.45,-7.075%,"1,881,588,201",/market-activity/stocks/sos,-0.07075,Software—Infrastructure,Technology
2160,IRNT,"IronNet, Inc. Common Stock",$0.18,-0.0097,-5.113%,"20,119,577",/market-activity/stocks/irnt,-0.05113,Software—Infrastructure,Technology
1547,TUYA,"Tuya Inc. American Depositary Shares, each rep...",$1.69,-0.08,-4.52%,"977,743,686",/market-activity/stocks/tuya,-0.04520,Software—Infrastructure,Technology
1570,GB,Global Blue Group Holding AG Ordinary Shares,$4.92,-0.17,-3.34%,"933,478,980",/market-activity/stocks/gb,-0.03340,Software—Infrastructure,Technology
1792,BKKT,"Bakkt Holdings, Inc. Class A Common Stock",$1.77,-0.05,-2.747%,"495,275,332",/market-activity/stocks/bkkt,-0.02747,Software—Infrastructure,Technology
1127,TIXT,TELUS International (Cda) Inc. Subordinate Vot...,$9.93,-0.17,-1.683%,"2,710,890,000",/market-activity/stocks/tixt,-0.01683,Software—Infrastructure,Technology
2142,VHC,VirnetX Holding Corp Common Stock,$0.4727,-0.0043,-0.901%,"33,762,432",/market-activity/stocks/vhc,-0.00901,Software—Infrastructure,Technology
637,WEX,WEX Inc. common stock,$192.89,-0.77,-0.398%,"8,263,538,958",/market-activity/stocks/wex,-0.00398,Software—Infrastructure,Technology
14,ORCL,Oracle Corporation Common Stock,$118.89,-0.38,-0.319%,"320,557,519,170",/market-activity/stocks/orcl,-0.00319,Software—Infrastructure,Technology
1426,PAY,"Paymentus Holdings, Inc. Class A Common Stock",$10.76,-0.03,-0.278%,"1,327,274,234",/market-activity/stocks/pay,-0.00278,Software—Infrastructure,Technology


In [ ]:
# @title Sharp drops
days_ago = 50 #@param {type:"slider", min:1, max:500, step:1}

test_sym = ["NVDA", "GOOG", "MSFT", "NET", "VSTM"] # DEBUG

# print(f"Given the last {days_ago} days of price data")

for index, symbol in nasdaq_df.iterrows():
  stock_symbol: str = symbol['symbol']
  stock_name: str = symbol['name']

  # if stock_symbol in test_sym: # DEBUG
  symbol_data: pd.DataFrame = get_symbol_data(stock_symbol, days_ago=90)
  if_spike: bool = has_large_drop_last3days(symbol_data)
  if if_spike:
    display(md(f"[{stock_name}](https://www.google.com/search?q={urllib.parse.quote(stock_name)}) has spike {if_spike}"))
    display(symbol_show_line(symbol_data, title=stock_name))

In [ ]:
# @title Weights assigned

weights = {
    "EARNINGS_BEAT_LAST_2": 20,
    "EARNINGS_BEAT_LAST_1": 20,
    "DROP_LAST_5_DAYS": 30,
    "UNASSIGNED": 30
}

sum = 0
for weight in weights.values():
  sum += weight

assert sum == 100, "Does not add up to 100"

In [ ]:
# @title Rating algorithms

def rate_income_statement(income_stmt_df: pd.DataFrame) -> float:
  return income_stmt_df["grossProfit"].astype("int").sort_values().pct_change().sum() * 100

# Fundamentals of company health

In [ ]:
# @markdown Limited financialmodelingprep.com (250/day)
choose_symbol = "MXL" #@param {type:"string"}

choose_symbol = clean_symbol(choose_symbol)

earnings_beat_df: pd.DataFrame = get_earnings_surprises(choose_symbol)
earnings_beat_chart(earnings_beat_df, choose_symbol)

alt.LayerChart(...)

In [ ]:
# @title Ratio comparison
# @markdown *Limit 5 calls / min*
many_symbols = "MXL,CRUS,AMBA,SMTC,MTSI" #@param {type:"string"}

clean_symbols_input = clean_symbols_many(many_symbols)

mixed_ratios_df_list: list[pd.DataFrame] = []

for symbol in clean_symbols_input:
  ratio: pd.DataFrame = get_ratios_ttm(symbol)
  ratio["symbol"] = symbol
  mixed_ratios_df_list.append(ratio)

mixed_ratios: pd.DataFrame = pd.concat(mixed_ratios_df_list, ignore_index=True)
melted_df: pd.DataFrame = pd.melt(mixed_ratios, id_vars=["symbol"], var_name="metric")

melted_filtered_df: pd.DataFrame = melted_df[
    melted_df["metric"].isin([
        "assetTurnoverTTM",
        "cashRatioTTM",
        "debtRatioTTM",
        "debtEquityRatioTTM",
        "pegRatioTTM",
        "freeCashFlowOperatingCashFlowRatioTTM",
        "payablesTurnoverTTM",
        "priceToSalesRatioTTM",
        "peRatioTTM"
        ])]

In [ ]:
# @title Get chart

def ratio_chart(ratio_df: pd.DataFrame, symbol_name: str = ""):
  if symbol_name != "":
    symbol_name = f"({symbol_name})"

  point = alt.OverlayMarkDef(filled=False, fill="white")

  expected_chart = alt.Chart(ratio_df).mark_point(size=200, strokeWidth=6).encode(
      row=alt.Row("metric:N",
                  header=alt.Header(labelAngle=0, labelFontSize=15)),
      x=alt.X("value:Q"),
      y=alt.Y("symbol:N", axis=None),
      tooltip=alt.Tooltip(["symbol:N", "value:Q", "metric:N"]),
      color=alt.Color("symbol:N"),
  # ).transform_window(
  #     groupby="metric",
  #     center="value"
  ).properties(
      title=f"Ratios {symbol_name}",
      width=1100,
  )

  return (expected_chart).configure_axis(
    labelFontSize=18,
    titleFontSize=18,
  ).configure_legend(
      labelFontSize=15,
      titleFontSize=12
  ).configure_title(
      fontSize=20
  ).configure_point(
      size=200,
  )

ratio_chart(melted_filtered_df)

alt.Chart(...)